# **LaTeX table with the BAO-fit results**

This notebook shows how to create a LaTeX table with the BAO-fit results from a set of mocks. It also makes some plots

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from itertools import product
import os
from utils_data import RedshiftDistributions
from utils_baofit import BAOFitInitializer

# # Option A. DESY6-related stuff
# dataset = "DESY6_COLA"
# dataset = "DESY6_COLA_DR1tiles_noDESI"
# dataset = "DESY6_COLA_DR1tiles_DESIonly"
# dataset = "DESY6_COLA_dec<-23.5"
# dataset = "DESY6_COLA_dec>-23.5"

# Option B. DESI-related stuff
dataset = "DESIY1_LRG_Abacus_altmtl_deltaz0.028"
# dataset = "DESIY1_LRG_EZ_ffa_deltaz0.028"

# dataset = "DESIY1_LRG_Abacus_complete_deltaz0.028"
# dataset = "DESIY1_LRG_EZ_complete_deltaz0.028"

if "DESIY1" in dataset:
    nz_flag = "mocks"
    cov_type = "mocks"
    cosmology_template = "desifid"
    cosmology_covariance = "desifid"

    if "EZ" in dataset:
        n_mocks = 1000
    elif "Abacus" in dataset:
        n_mocks = 25
    
elif "DESY6" in dataset:
    nz_flag = "mocks"
    cov_type = "cosmolike"
    cosmology_template = "mice"
    cosmology_covariance = "mice"
    
    n_mocks = 1952

mock_id_list = range(n_mocks)
mock_id_detected_list = []
    
fit_results = {}

for mock_id in mock_id_list:
    # 1. Arguments
    class Args:
        def __init__(self):
            self.include_wiggles = "y"
            self.dataset = dataset
            self.weight_type = 1 # it will not be used...
            self.mock_id = mock_id
            self.nz_flag = nz_flag
            self.cov_type = cov_type
            self.cosmology_template = cosmology_template
            self.cosmology_covariance = cosmology_covariance
            self.delta_theta = 0.2
            self.theta_min = None
            self.theta_max = None
            self.n_broadband = 3
            self.bins_removed = []
            self.diag_only = "n"
            self.remove_crosscov = "n"
            self.alpha_min = 0.8
            self.alpha_max = 1.2
            self.base_path = None
    args = Args()
    args.include_wiggles = "" if args.include_wiggles == "y" else "_nowiggles"
    
    # 2. BAO fit initializer. This basically creates the path to load the results
    baofit_initializer = BAOFitInitializer(
        include_wiggles=args.include_wiggles,
        dataset=args.dataset,
        weight_type=args.weight_type,
        mock_id=args.mock_id,
        nz_flag=args.nz_flag,
        cov_type=args.cov_type,
        cosmology_template=args.cosmology_template,
        cosmology_covariance=args.cosmology_covariance,
        delta_theta=args.delta_theta,
        theta_min=args.theta_min,
        theta_max=args.theta_max,
        n_broadband=args.n_broadband,
        bins_removed=args.bins_removed,
        alpha_min=args.alpha_min,
        alpha_max=args.alpha_max,
        verbose=False,
        base_path=args.base_path,
    )

    try:
        results = np.loadtxt(os.path.join(baofit_initializer.path_baofit, "fit_results.txt"))
        if not os.path.exists(os.path.join(baofit_initializer.path_baofit, "wtheta_data_bestfit.txt")):
            print(f"Mock number {mock_id} has a non-detection!")
        else:
            fit_results[mock_id] = results
            mock_id_detected_list.append(mock_id)
            
    except:
        break
        
print(f"Last mock available: {mock_id}")


In [ ]:
import pandas as pd

data = []
for key, values in fit_results.items():
    mock_id = key
    alpha, sigma_alpha, chi2, dof = values
    data.append([mock_id, alpha, sigma_alpha, chi2, int(dof)])

df = pd.DataFrame(data, columns=["Mock id", r"$\alpha$", r"$\sigma_\alpha$", r"$\chi^2$", "dof"])

df[r"$\alpha$"] = df[r"$\alpha$"].map(lambda x: f"{x:.3f}")
df[r"$\sigma_\alpha$"] = df[r"$\sigma_\alpha$"].map(lambda x: f"{x:.3f}")
df[r"$\chi^2$"] = df[r"$\chi^2$"].map(lambda x: f"{x:.1f}")
df["dof"] = df["dof"].astype(int)

latex_table = df.to_latex(index=False, escape=False, column_format="c|cccc")


In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import chi2

# Define figure size
figsize = (7, 5)

# Compute means
mean_alpha = np.mean([fit_results[mock_id][0] for mock_id in mock_id_detected_list])
mean_sigma_alpha = np.mean([fit_results[mock_id][1] for mock_id in mock_id_detected_list])
mean_chi2 = np.mean([fit_results[mock_id][2] for mock_id in mock_id_detected_list])

# Histogram for alpha
plt.figure(figsize=figsize)
plt.hist([fit_results[mock_id][0] for mock_id in mock_id_detected_list], color='royalblue', edgecolor='black', alpha=0.7)
plt.axvline(mean_alpha, color='black', linestyle="--", linewidth=2, label=fr'$\langle\alpha\rangle$ = {mean_alpha:.4f}')
plt.xlabel(r'$\alpha$', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(fontsize=14)

# Histogram for sigma_alpha
plt.figure(figsize=figsize)
plt.hist([fit_results[mock_id][1] for mock_id in mock_id_detected_list], color='darkorange', edgecolor='black', alpha=0.7)
plt.axvline(mean_sigma_alpha, color='black', linestyle="--", linewidth=2, label=fr'$\langle\sigma_\alpha\rangle$ = {mean_sigma_alpha:.4f}')
plt.xlabel(r'$\sigma_{\alpha}$', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(fontsize=14)

# Histogram for chi^2
plt.figure(figsize=figsize)
plt.hist([fit_results[mock_id][2] for mock_id in mock_id_detected_list], density=True, color='forestgreen', edgecolor='black', alpha=0.7)
plt.axvline(mean_chi2, color='black', linestyle="--", linewidth=2, label=fr'$\langle\chi^2\rangle$ = {mean_chi2:.1f}')
plt.xlabel(r'$\chi^2$', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# Overlay chi-squared distribution
dof = fit_results[0][3]
x = np.linspace(0, dof * 2, 500)
pdf = chi2.pdf(x, dof)
plt.plot(x, pdf, label=fr'$\chi^2$ dist. (dof = {int(dof)})', color='darkgreen')
# plt.xlim([350, 550])
# plt.xlim([50, 160])
plt.legend(loc='upper right', fontsize=14)

# Print means
print(mean_alpha)
print(mean_sigma_alpha)
print([mean_chi2, dof])
